# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment, Dataset
from azureml.train.automl import AutoMLConfig

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'Capstone-AutoML'

exp=Experiment(ws, experiment_name)
run=exp.start_logging()

In [3]:
#create compute cluster
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "proj3cluster"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
# pima diabetes dataset
train_ds = Dataset.get_by_name(ws, name='pima-diabetes')


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
# TODO: Put your automl settings here
automl_settings = {}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_ds,
    label_column_name='Outcome',
    n_cross_validations=5,
    compute_target = cpu_cluster)

In [7]:
# TODO: Submit your experiment
remote_run = exp.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
Capstone-AutoML,AutoML_6fbd8108-694c-4c0f-b574-162150ee33a4,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
from azureml.widgets import RunDetails
# Monitor the run using RunDetails widget
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [10]:
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

Package:azureml-automl-runtime, training version:1.50.0, current version:1.49.0
Package:azureml-core, training version:1.50.0, current version:1.49.0
Package:azureml-dataprep, training version:4.10.6, current version:4.9.1
Package:azureml-dataprep-rslex, training version:2.17.5, current version:2.16.1
Package:azureml-dataset-runtime, training version:1.50.0, current version:1.49.0
Package:azureml-defaults, training version:1.50.0, current version:1.49.0
Package:azureml-interpret, training version:1.50.0, current version:1.49.0
Package:azureml-mlflow, training version:1.50.0, current version:1.49.0
Package:azureml-pipeline-core, training version:1.50.0.post1, current version:1.49.0
Package:azureml-responsibleai, training version:1.50.0, current version:1.49.0
Package:azureml-telemetry, training version:1.50.0, current version:1.49.0
Package:azureml-train-automl-client, training version:1.50.0, current version:1.49.0
Package:azureml-train-automl-runtime, training version:1.50.0, current 

Run(Experiment: Capstone-AutoML,
Id: AutoML_6fbd8108-694c-4c0f-b574-162150ee33a4_37,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('26', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=True)), ('logisticregression', LogisticRegression(C=1.7575106248547894, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='multinomial', n_jobs=1, penalty='l1', random_state=None, solver='saga', tol=

In [11]:
#TODO: Save the best model
import joblib
joblib.dump(fitted_model,'automl_best_model.pkl')



['automl_best_model.pkl']

In [41]:

best_run.register_model(model_name='best_run_capstone_automl', model_path='./outputs/')

Model(workspace=Workspace.create(name='quick-starts-ws-233997', subscription_id='9e65f93e-bdd8-437b-b1e8-0647cd6098f7', resource_group='aml-quickstarts-233997'), name=best_run_capstone_automl, id=best_run_capstone_automl:1, version=1, tags={}, properties={})

In [46]:
#download environment file
best_run.download_file('outputs/conda_env_v_1_0_0.yml')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [43]:
from azureml.core.model import Model
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
model = remote_run.register_model(model_name = 'best_model_automl')

environment = best_run.get_environment()
entry_script='testing/scoring.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', entry_script)


inference_config = InferenceConfig(entry_script = entry_script, environment = environment)

deployment_config = AciWebservice.deploy_configuration( auth_enabled= True, 
                                                    enable_app_insights= True)

service = Model.deploy(ws, "automl-best-model1", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-05-19 09:55:43+00:00 Creating Container Registry if not exists.
2023-05-19 09:55:43+00:00 Registering the environment.
2023-05-19 09:55:44+00:00 Use the existing image.
2023-05-19 09:55:44+00:00 Submitting deployment to compute.
2023-05-19 09:55:49+00:00 Checking the status of deployment automl-best-model1..
2023-05-19 09:58:13+00:00 Checking the status of inference endpoint automl-best-model1.
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, send a request to the web service you deployed to test it.

In [50]:
from azureml.core import Webservice
import json


# Specify the deployed service name
service_name = 'automl-best-model1'

# Get the deployed web service
service = Webservice(ws, name=service_name)


input_data = {
    'data': [
        {'Pregnancies': 6, 'Glucose': 8, 'BloodPressure':72, 'SkinThickness':35, 'Insulin':0,
    'BMI':33.6,'DiabetesPedigreeFunction':0.627,'Age':50}  
    ]
}
# Convert input data to JSON format
input_data_json = json.dumps(input_data)

# Send a request to the web service
output = service.run(input_data_json)

# Print the predicted output
print(output)


{"result": [0]}


TODO: In the cell below, print the logs of the web service and delete the service

In [51]:
from azureml.core import Webservice
# Specify the deployed service name
service_name = 'automl-best-model1'

# Get the deployed web service
service = Webservice(ws, name=service_name)

# Get the deployment logs
logs = service.get_logs()

# Print the deployment logs
print(logs)


2023-05-19T09:57:58,044920200+00:00 - rsyslog/run 
2023-05-19T09:57:58,061765900+00:00 - gunicorn/run 
2023-05-19T09:57:58,070843600+00:00 | gunicorn/run | 
2023-05-19T09:57:58,105668600+00:00 - nginx/run 
2023-05-19T09:57:58,110110400+00:00 | gunicorn/run | ###############################################
2023-05-19T09:57:58,125072500+00:00 | gunicorn/run | AzureML Container Runtime Information
2023-05-19T09:57:58,126561100+00:00 | gunicorn/run | ###############################################
2023-05-19T09:57:58,132219100+00:00 | gunicorn/run | 
2023-05-19T09:57:58,202883300+00:00 | gunicorn/run | 
2023-05-19T09:57:58,214434800+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20230412.v4
2023-05-19T09:57:58,215901400+00:00 | gunicorn/run | 
2023-05-19T09:57:58,217266300+00:00 | gunicorn/run | 
2023-05-19T09:57:58,222546900+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/bin:/usr/local/sbi

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
